# Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras

# Set image size, batch size, and import dataset

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

train = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-03 15:55:31.145370: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 15:55:31.145512: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Train from the 4th conv2d layer

In [3]:
vtid = keras.models.load_model('../../models/vtid2/m1_big.h5')
vtid.trainable = False

for layer in vtid.layers[6:]:
  layer.trainable = True

x = vtid.layers[-2].output
outputs = layers.Dense(6, activation='softmax', name='classifier')(x)
model = keras.Model(vtid.inputs, outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 94, 94, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 47, 47, 32)       0         
 )                                                               
                                                             

In [4]:

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/transfer/transfer_big_trainHalf.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]


In [5]:
history = model.fit(
  train.cache(),
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/50


2022-08-03 15:55:31.850029: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 15:55:31.850133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.1043 - accuracy: 0.5946

2022-08-03 15:55:38.928583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 8s 71ms/step - loss: 1.1043 - accuracy: 0.5946 - val_loss: 0.5841 - val_accuracy: 0.7806
Epoch 2/50
105/105 [==============================] - 7s 64ms/step - loss: 0.5562 - accuracy: 0.8027 - val_loss: 0.4694 - val_accuracy: 0.8500
Epoch 3/50
105/105 [==============================] - 7s 62ms/step - loss: 0.3364 - accuracy: 0.8863 - val_loss: 0.3907 - val_accuracy: 0.8639
Epoch 4/50
105/105 [==============================] - 7s 65ms/step - loss: 0.2114 - accuracy: 0.9307 - val_loss: 0.5807 - val_accuracy: 0.8639
Epoch 5/50
105/105 [==============================] - 7s 66ms/step - loss: 0.1730 - accuracy: 0.9423 - val_loss: 0.3869 - val_accuracy: 0.9014
Epoch 6/50
105/105 [==============================] - 7s 65ms/step - loss: 0.1276 - accuracy: 0.9563 - val_loss: 0.3478 - val_accuracy: 0.8986
Epoch 7/50
105/105 [==============================] - 7s 64ms/step - loss: 0.1093 - accuracy: 0.9649 - val_loss: 0.4980 - val_accuracy: 0.8750
Epoch 8/50

In [6]:
model = keras.models.load_model("../../models/transfer/transfer_big_trainHalf.h5")
model.evaluate(test)

 3/23 [==>...........................] - ETA: 0s - loss: 0.3585 - accuracy: 0.9062

2022-08-03 15:57:45.135908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 32ms/step - loss: 0.3892 - accuracy: 0.9181


[0.3892287313938141, 0.9180555939674377]

# compare with untrained

In [7]:
untrained_model = keras.models.load_model("../../models/zenodo/m1_big.h5")
untrained_model.evaluate(test)

 3/23 [==>...........................] - ETA: 0s - loss: 0.0613 - accuracy: 0.9688

2022-08-03 15:57:46.191352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 31ms/step - loss: 0.3322 - accuracy: 0.9597


[0.33215728402137756, 0.9597222208976746]

# All Trainable Layers Transfer Learning - start from nonrandom weights

In [8]:
vtid2_allTrainable = keras.models.load_model('../../models/vtid2/m1_big.h5')

x = vtid2_allTrainable.layers[-2].output
outputs = keras.layers.Dense(6, activation='softmax', name='classifier')(x)
all_trainable_model = keras.Model(vtid2_allTrainable.inputs, outputs)
all_trainable_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
all_trainable_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 94, 94, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 47, 47, 32)       0         
 )                                                               
                                                           

In [9]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/transfer/transfer_big_trainFull.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

In [10]:
history = all_trainable_model.fit(
  train.cache(),
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/50


2022-08-03 15:57:47.641399: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.1525 - accuracy: 0.5670

2022-08-03 15:57:57.394141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 11s 95ms/step - loss: 1.1525 - accuracy: 0.5670 - val_loss: 0.6888 - val_accuracy: 0.7417
Epoch 2/50
105/105 [==============================] - 10s 93ms/step - loss: 0.5608 - accuracy: 0.7994 - val_loss: 0.4517 - val_accuracy: 0.8389
Epoch 3/50
105/105 [==============================] - 10s 94ms/step - loss: 0.3561 - accuracy: 0.8759 - val_loss: 0.3787 - val_accuracy: 0.8750
Epoch 4/50
105/105 [==============================] - 10s 92ms/step - loss: 0.2602 - accuracy: 0.9122 - val_loss: 0.3226 - val_accuracy: 0.9028
Epoch 5/50
105/105 [==============================] - 10s 93ms/step - loss: 0.2028 - accuracy: 0.9417 - val_loss: 0.4023 - val_accuracy: 0.8875
Epoch 6/50
105/105 [==============================] - 10s 93ms/step - loss: 0.1692 - accuracy: 0.9512 - val_loss: 0.2799 - val_accuracy: 0.9181
Epoch 7/50
105/105 [==============================] - 10s 94ms/step - loss: 0.1273 - accuracy: 0.9619 - val_loss: 0.3239 - val_accuracy: 0.9194
Epo

In [11]:
trainable_model = keras.models.load_model("../../models/transfer/transfer_big_trainFull.h5")
trainable_model.evaluate(test)

 1/23 [>.............................] - ETA: 5s - loss: 0.3203 - accuracy: 0.9375

2022-08-03 16:01:42.332459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 36ms/step - loss: 0.3340 - accuracy: 0.9403


[0.3339744508266449, 0.9402778148651123]